In [1]:
import pandas as pd 
import os
pen_info= pd.concat([pd.read_csv(os.path.join('/home/rajesh/work/failed_milkings/events_data',file)) for file in os.listdir('/home/rajesh/work/failed_milkings/events_data') if file.startswith('events')])
failed_df= pd.read_csv('/home/rajesh/work/failed_milkings/final_df_with_days_relative.csv')
pen_info= pen_info[pen_info['PEN'].isin([30,31,32,33,34,35,36])]
pen_info.PEN.value_counts()
pen_info['Date']=pd.to_datetime(pen_info['Date'], format='mixed')

pen_info['Animal Number']=pen_info['ID'].astype(int)

failed_df['Animal Number']= failed_df['Animal Number'].astype(int)

failed_df['Date Time']= pd.to_datetime(failed_df['Date Time'])

failed_df = failed_df.reset_index(drop=True)
pen_info = pen_info.reset_index(drop=True)


import pandas as pd
import numpy as np
from multiprocessing import Pool, cpu_count

# --- Ensure clean datetime and consistent Animal Number type ---
failed_df['Date Time'] = pd.to_datetime(failed_df['Date Time'])
pen_info['Date'] = pd.to_datetime(pen_info['Date'])
failed_df['Animal Number'] = failed_df['Animal Number'].astype(str)
pen_info['Animal Number'] = pen_info['Animal Number'].astype(str)

# --- Create a lookup dict for pen_info grouped by Animal Number ---
pen_groups = {k: v for k, v in pen_info.groupby('Animal Number')}

# --- Function to find the nearest match for one row ---
def find_nearest(row):
    animal = row['Animal Number']
    event_time = row['Date Time']
    
    # If animal not in pen_info, return row unchanged (with NaNs for pen columns)
    if animal not in pen_groups:
        return {**row.to_dict(), 'Matched Date': pd.NaT}
    
    df_animal = pen_groups[animal]
    # Find index of nearest date
    nearest_idx = (df_animal['Date'] - event_time).abs().idxmin()
    nearest_row = df_animal.loc[nearest_idx].to_dict()
    
    # Merge dictionaries
    merged = {**row.to_dict(), **nearest_row}
    merged['Matched Date'] = nearest_row['Date']
    return merged

# --- Run in parallel across all CPU cores ---
num_cores = max(cpu_count() - 1, 1)  # leave one core free
with Pool(num_cores) as pool:
    merged_records = pool.map(find_nearest, [row for _, row in failed_df.iterrows()])

# --- Combine into final DataFrame ---
merged_df = pd.DataFrame(merged_records)

print("✅ Parallel nearest-date merge completed successfully!")
print(merged_df.head())



/tmp/ipykernel_2766/3707399845.py:4: DtypeWarning: Columns (6,49,74,75) have mixed types. Specify dtype option on import or set low_memory=False.
  failed_df= pd.read_csv('/home/rajesh/work/failed_milkings/final_df_with_days_relative.csv')


✅ Parallel nearest-date merge completed successfully!
   Unnamed: 0.2  Unnamed: 0.1 Animal Number  Unnamed: 1  Unnamed: 0  \
0             0             1             3           1           1   
1             1             2             3           2           2   
2             2             3             3           3           3   
3             3             4             3           4           4   
4             4             5             3           5           5   

   Animal Number.1 Gestation  Animal Name  Animal Tag Id  Conductivity LR  \
0                3         3          NaN      6697303.0             66.0   
1                3         3          NaN      6697303.0             65.0   
2                3         3          NaN      6697303.0             67.0   
3                3         3          NaN      6697303.0             64.0   
4                3         3          NaN      6697303.0             63.0   

   ...   DIM       Date            Remark   R  T   B    

In [10]:
merged_df.groupby(['Device Name', 'PEN']).size().reset_index(name='Count').sort_values(['Device Name', 'Count'], ascending=[True, False]).drop_duplicates('Device Name')


,Device Name,PEN,Count
0,Astronaut 101,30.0,24734
6,Astronaut 102,30.0,31616
12,Astronaut 103,30.0,29975
19,Astronaut 104,31.0,29319
25,Astronaut 105,31.0,28176
31,Astronaut 106,31.0,26098
38,Astronaut 107,32.0,29534
44,Astronaut 108,32.0,26463
50,Astronaut 109,32.0,25747
57,Astronaut 110,33.0,39238


In [19]:
merged_df_filtered= merged_df[merged_df['Device Name'].isin(
    ['Astronaut 107', 'Astronaut 109', 'Astronaut 108', 'Astronaut 106',
       'Astronaut 105', 'Astronaut 104', 'Astronaut 110', 'Astronaut 111',
       'Astronaut 112', 
       'Astronaut 103', 'Astronaut 102', 'Astronaut 101', 'Astronaut 118', 'Astronaut 117',
        'Astronaut 116',
       'Astronaut 113',
       'Astronaut 115', 
       'Astronaut 114'])
      ]

In [32]:
merged_df_filtered['correct_pen'] = merged_df_filtered['Device Name'].apply(
    lambda x: 30 if x.endswith(('101', '102', '103'))
    else 31 if x.endswith(('104', '105', '106'))
    else 32 if x.endswith(('107', '108', '109'))
    else 33 if x.endswith(('110', '111', '112'))
    else 34 if x.endswith(('113', '114', '115'))
    else 35
)


/tmp/ipykernel_2766/1935382931.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_df_filtered['correct_pen'] = merged_df_filtered['Device Name'].apply(


In [34]:
merged_df_filtered.to_csv('/home/rajesh/work/failed_milkings/corrected_pen_df.csv')

In [50]:
merged_df_filtered['Device Name'].value_counts()

Device Name
Astronaut 111    43062
Astronaut 115    42896
Astronaut 114    42314
Astronaut 110    42143
Astronaut 112    41840
Astronaut 113    39146
Astronaut 118    34258
Astronaut 102    34089
Astronaut 103    33099
Astronaut 107    32536
Astronaut 104    31038
Astronaut 117    29919
Astronaut 105    29806
Astronaut 116    29391
Astronaut 106    28760
Astronaut 108    28711
Astronaut 101    28502
Astronaut 109    28224
Name: count, dtype: int64

(1|anial_id)+ (pen/robot_id)

In [ ]:
(1|Cow) + (1|robot:Pen)+ pen , 
 mean + variance 


In [44]:
merged_df_filtered['Visit Result']= merged_df_filtered['Visit Result'].apply(lambda x: 'Other' if x in ['Failed', 'Teats not found', 'Robot arm stopped'] else x)
merged_df_filtered['visit_results_new']= merged_df_filtered['Visit Result'].apply(lambda x: 'Connection attempt' if x.startswith('Connection attemp') else ('Dead milk time' if x.startswith('Dead milk') else x) )

/tmp/ipykernel_2766/595562160.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_df_filtered['Visit Result']= merged_df_filtered['Visit Result'].apply(lambda x: 'Other' if x in ['Failed', 'Teats not found', 'Robot arm stopped'] else x)
/tmp/ipykernel_2766/595562160.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_df_filtered['visit_results_new']= merged_df_filtered['Visit Result'].apply(lambda x: 'Connection attempt' if x.startswith('Connection attemp') else ('Dead milk time' if x.startswi

In [47]:
merged_df_filtered['weeks']= merged_df_filtered['Lactation Days'].astype(int).apply(lambda x: round(x/7)+1)
merged_df_filtered['seasons_milking']= pd.to_datetime(merged_df_filtered['Date Time']).dt.month.apply(lambda x: 'winter' if x in [12, 1, 2] else ('spring' if x in [3, 4, 5] else ('summer' if x in [6, 7, 8] else 'fall')))
merged_df_filtered['parity']= merged_df_filtered['Gestation'].astype(str).apply(lambda x: 'Primiparous' if x.strip()=='1' else 'Multiparous')
merged_df_filtered['gestation']= merged_df_filtered['Gestation'].astype(str)

df_followup = (
    merged_df_filtered
    .groupby(['Animal Number', 'gestation'], as_index=False)
    .agg(followup_weeks=('weeks', lambda x: x.max(skipna=True) - x.min(skipna=True) + 1))
)

merged_df_filtered = pd.merge(
    merged_df_filtered,
    df_followup,
    on=['Animal Number', 'gestation'],
    how='left'
)


# Step 2: Aggregate binary failure merged_df_filtered into counts per group
agg_counts = (
    merged_df_filtered.groupby(['weeks', 'parity', 'visit_results_new'], as_index=False)
        .agg(
            failed_count=('failed', 'sum'),
            total_obs=('failed', 'count'),
            avg_followup_weeks=('followup_weeks', 'mean')  # optional
        )
)

/tmp/ipykernel_2766/1095939898.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_df_filtered['weeks']= merged_df_filtered['Lactation Days'].astype(int).apply(lambda x: round(x/7)+1)
/tmp/ipykernel_2766/1095939898.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_df_filtered['seasons_milking']= pd.to_datetime(merged_df_filtered['Date Time']).dt.month.apply(lambda x: 'winter' if x in [12, 1, 2] else ('spring' if x in [3, 4, 5] else ('summer' if x in [6, 7, 8] else 'fall')))
/tmp/ipykernel_27

In [59]:
merged_df_filtered.shape

(619734, 96)

In [62]:
merged_df_filtered['Date Time']

0        2023-04-25 06:25:00
1        2023-04-25 13:55:00
2        2023-04-25 22:08:00
3        2023-04-26 06:11:00
4        2023-04-26 14:32:00
                 ...        
619729   2024-06-29 04:27:00
619730   2024-06-29 11:46:00
619731   2024-06-30 00:37:00
619732   2024-06-30 10:16:00
619733   2024-06-30 17:39:00
Name: Date Time, Length: 619734, dtype: datetime64[ns]

In [53]:
agg_counts

,weeks,parity,visit_results_new,failed_count,total_obs,avg_followup_weeks
0,1,Multiparous,Connection attempt,2.0,2,39.500000
1,1,Multiparous,Connection time,5.0,5,36.200000
2,1,Multiparous,Dead milk time,3.0,3,5.000000
3,1,Multiparous,Successful,0.0,1010,42.988119
4,2,Multiparous,Automatic robot stop,4.0,4,43.250000
...,...,...,...,...,...,...
340,75,Multiparous,Successful,0.0,23,67.217391
341,76,Multiparous,Connection time,4.0,4,77.000000
342,76,Multiparous,Dead milk time,9.0,9,77.000000
343,76,Multiparous,Successful,0.0,13,77.000000


In [56]:
weekly_frequency= merged_df_filtered.groupby([ 'Lactation Number', 'weeks', 'visit_results_new'])['failed'].sum().reset_index()
weekly_frequency['Lactation_Number']= weekly_frequency['Lactation Number'].apply(lambda x: 'First' if x==1 else 'Multiparous')

In [57]:
weekly_frequency

,Lactation Number,weeks,visit_results_new,failed,Lactation_Number
0,1,1,Connection attempt,1.0,First
1,1,1,Connection time,1.0,First
2,1,1,Successful,0.0,First
3,1,2,Connection attempt,12.0,First
4,1,2,Connection time,8.0,First
...,...,...,...,...,...
1215,7,30,Successful,0.0,Multiparous
1216,7,31,Successful,0.0,Multiparous
1217,7,32,Successful,0.0,Multiparous
1218,7,33,Successful,0.0,Multiparous
